In [1]:
# !wget -O "sentiment.zip" "http://thinknook.com/wp-content/uploads/2012/09/Sentiment-Analysis-Dataset.zip"

In [2]:
# !unzip sentiment.zip

In [3]:
# !mv Sentiment\ Analysis\ Dataset.csv data.csv

In [4]:
import numpy as np
import pandas as pd

training = pd.read_csv('data.csv', error_bad_lines=False, usecols=["Sentiment", "SentimentText"])

In [5]:
training.head()

,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...


In [6]:
train_x = [x for x in training.SentimentText]
train_y = np.asarray([x for x in training.Sentiment])

In [7]:
import json
import tensorflow.keras as keras
import tensorflow.keras.preprocessing.text as kpt
from tensorflow.keras.preprocessing.text import Tokenizer

# only work with the 3000 most popular words found in our dataset
max_words = 3000

# create a new Tokenizer
tokenizer = Tokenizer(num_words=max_words)
# feed our tweets to the Tokenizer
tokenizer.fit_on_texts(train_x)

# Tokenizers come with a convenient list of words and IDs
dictionary = tokenizer.word_index
# Let's save this out so we can use it later
with open('dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)

In [8]:
def convert_text_to_index_array(text):
    # one really important thing that `text_to_word_sequence` does
    # is make all texts the same length -- in this case, the length
    # of the longest text in the set.
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

In [9]:
allWordIndices = []
# for each tweet, change each token to its ID in the Tokenizer's word_index
for text in train_x:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

In [10]:
allWordIndices = np.asarray(allWordIndices)

# create one-hot matrices out of the indexed tweets
train_x = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
# treat the labels as categories
train_y = keras.utils.to_categorical(train_y, 2)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation

model = Sequential()
model.add(Dense(512, input_shape=(max_words,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [12]:
model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])

In [13]:
history = model.fit(train_x, train_y,
    batch_size=10000,
    epochs=10,
    verbose=1,
    validation_split=0.1,
    shuffle=True)

Train on 1420752 samples, validate on 157862 samples
Epoch 1/10
1420752/1420752 [==============================] - 126s 89us/sample - loss: 0.5319 - accuracy: 0.7328 - val_loss: 0.4320 - val_accuracy: 0.8033
Epoch 2/10
1420752/1420752 [==============================] - 132s 93us/sample - loss: 0.4518 - accuracy: 0.7864 - val_loss: 0.4169 - val_accuracy: 0.8103
Epoch 3/10
1420752/1420752 [==============================] - 136s 96us/sample - loss: 0.4396 - accuracy: 0.7935 - val_loss: 0.4077 - val_accuracy: 0.8149
Epoch 4/10
1420752/1420752 [==============================] - 136s 96us/sample - loss: 0.4296 - accuracy: 0.7994 - val_loss: 0.4006 - val_accuracy: 0.8188
Epoch 5/10
1420752/1420752 [==============================] - 136s 96us/sample - loss: 0.4209 - accuracy: 0.8046 - val_loss: 0.3952 - val_accuracy: 0.8213
Epoch 6/10
1420752/1420752 [==============================] - 137s 96us/sample - loss: 0.4137 - accuracy: 0.8085 - val_loss: 0.3928 - val_accuracy: 0.8228
Epoch 7/10
142075

In [14]:
model.save('sentiment_model.h5')